
# Not ready, should be tested!

In [ ]:

from pathlib import Path
import json
import shutil
import rasterio.shutil
from utils.preprocess.split_channels import split_channels

import sys
sys.path.append("/home/shamil/AILab/Rosreest")
sys.path.append("/home/shamil/AILab/artificial_cartographer")

In [ ]:
def clean_folder_content(folder: Path):
	"""Remove content of folder """
	if folder.is_dir():
		shutil.rmtree(folder)

	folder.mkdir()


f_threshold = {'f_score': 0.6}

In [ ]:
# Path to folder with inference images
inference_input_folder = Path('/home/shamil/pokaz_inference/для показа_13.08')
assert inference_input_folder.is_dir()

# Folder, where ot store tests
tests_folder = Path('/home/shamil/inference_pokaz/tests/data/pieceofland/quality/aerial/summer')
clean_folder_content(tests_folder)

# Folder, where results of running tests are stored
result_folder = Path('/home/shamil/inference_pokaz/tests/results/pieceofland/quality/aerial/summer')
clean_folder_content(result_folder)

# Folder for stashing results
output_folder = Path('/home/shamil/inference_output')
clean_folder_content(output_folder)

In [ ]:
# Inference folder contains sub folders, creating them in inference_output
folders = list(path.name for path in inference_input_folder.glob("**/*") if path.is_dir())

for name in folders:
	path = output_folder / name
	path.mkdir()


In [ ]:
# Preparing data for inference
for ii, path in enumerate(sorted(inference_input_folder.glob("**/*"))):
	if path.is_file():
		print(f"Processing: {str(path)}...")

		test = tests_folder / str(ii)
		test.mkdir()

		band = test / 'bands'
		band.mkdir()

		split_channels(_image_path=path, _dst=band)
		rasterio.shutil.copy(src=path, dst=test / "image.tif")

		ground_truth = test / 'ground_truth'
		ground_truth.mkdir()

		with open(test / "score_thresholds.json", 'w') as file:
			file.write(json.dumps(f_threshold))

In [1]:
# Running tests
! python /home/shamil/AILab/artificial_cartographer/tests/piece_of_land_detection/test_aerial.py

1


In [ ]:
# Renaming outputs
for ii, path in enumerate(sorted(inference_input_folder.glob("**/*"))):
	if path.is_file():
		shp_folder = tests_folder / str(ii) / "polygons"
		parent_name = path.parent.name

		# if file was included in folder -> write to there
		if parent_name in folders:
			output_path = output_folder / parent_name / path.name
		else:
			output_path = output_folder / path.name

		rasterio.shutil.copy(src=shp_folder, dst=output_path)
